In [27]:
%pip install scikit-learn==1.0.2

Note: you may need to restart the kernel to use updated packages.


In [28]:
import re
import sys
import pandas as pd
import pyttsx3
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier,_tree
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
import csv
import warnings

In [29]:
# pip install pyttsx3
# %pip install flask

In [30]:
# # prompt: code to know the version of dependencies

# import re
# import sys
# import pandas as pd
# import pyttsx3
# from sklearn import preprocessing
# from sklearn.tree import DecisionTreeClassifier,_tree
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import cross_val_score
# from sklearn.svm import SVC
# import csv
# import warnings

# print("Python version:", sys.version)

# # Use pip show to get the versions
# !pip show pandas | grep Version
# !pip show pyttsx3 | grep Version
# !pip show scikit-learn | grep Version # scikit-learn is the correct package name
# !pip show numpy | grep Version
# !pip show scikit-learn | grep Version # scikit-learn is the correct package name

# !pip show numpy | grep Version
# !pip show csv | grep Version # csv is a built-in module, so pip show won't work
# print("csv: built-in module")
# !pip show warnings | grep Version # warnings is a built-in module
# print("warnings: built-in module")


In [31]:
color = sys.stdout
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [32]:
training = pd.read_csv('Training.csv')
testing= pd.read_csv('Testing.csv')
cols= training.columns
cols= cols[:-1]
x = training[cols]
y = training['prognosis']
y1= y

In [33]:

reduced_data = training.groupby(training['prognosis']).max()

In [34]:
#mapping strings to numbers
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

In [35]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
testx    = testing[cols]
testy    = testing['prognosis']
testy    = le.transform(testy)

In [36]:
clf1  = DecisionTreeClassifier()
clf = clf1.fit(x_train.values,y_train)
# print(clf.score(x_train,y_train))
# print ("cross result========")
scores = cross_val_score(clf, x_test, y_test, cv=3)
# print (scores)
#print (scores.mean())

In [37]:


model=SVC()
model.fit(x_train.values,y_train)
#print("for svm: ")
#print(model.score(x_test,y_test))

SVC()

In [38]:
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
features = cols

In [39]:
def readn(nstr):
    engine = pyttsx3.init()

    engine.setProperty('voice', "english+f5")
    engine.setProperty('rate', 130)

    engine.say(nstr)
    engine.runAndWait()
    engine.stop()

In [40]:

severityDictionary=dict()
description_list = dict()
precautionDictionary=dict()

symptoms_dict = {}

for index, symptom in enumerate(x):
       symptoms_dict[symptom] = index

In [41]:

def calc_condition(exp,days):
    sum=0
    for item in exp:
        sum=sum+severityDictionary[item]
        #import pdb;pdb.set_trace()
    if((sum*days)/(len(exp)+1)>13):
        color.write("You should take the consultation from doctor.")#,"COMMENT")
        print()
    else:
        color.write("It might not be that bad but you should take precautions.")# ,"STRING")
        print()


In [42]:
def getDescription():
    global description_list
    with open('symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _description={row[0]:row[1]}
            description_list.update(_description)


In [43]:
def getSeverityDict():
    global severityDictionary
    with open('Symptom_severity.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        try:
            for row in csv_reader:
                _diction={row[0]:int(row[1])}
                severityDictionary.update(_diction)
        except:
            pass

In [44]:
def getprecautionDict():
    global precautionDictionary
    with open('symptom_precaution.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _prec={row[0]:[row[1],row[2],row[3],row[4]]}
            precautionDictionary.update(_prec)



In [45]:
def getInfo():
    print("-----------------------------------HealthCare ChatBot-----------------------------------")
    print("\nYour Name? \t\t\t\t",end="->")
    name=input("")
    print("Hello, ",name)

In [46]:

def check_pattern(dis_list,inp):
    pred_list=[]
    inp=inp.replace(' ','_')
    patt = f"{inp}"
    regexp = re.compile(patt)
    pred_list=[item for item in dis_list if regexp.search(item)]
    if(len(pred_list)>0):
        return 1,pred_list
    else:
        return 0,[]

In [47]:
def sec_predict(symptoms_exp):
    df = pd.read_csv('Training.csv')
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    rf_clf = DecisionTreeClassifier()
    rf_clf.fit(X_train.values, y_train)

    symptoms_dict = {symptom: index for index, symptom in enumerate(X)}
    input_vector = np.zeros(len(symptoms_dict))
    for item in symptoms_exp:
      input_vector[[symptoms_dict[item]]] = 1

    return rf_clf.predict([input_vector])

In [48]:

def print_disease(node):
    node = node[0]
    val  = node.nonzero()
    disease = le.inverse_transform(val[0])
    return list(map(lambda x:x.strip(),list(disease)))

In [49]:
def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    chk_dis=",".join(feature_names).split(",")
    symptoms_present = []

    while True:

        print("\nEnter the symptom you are experiencing  \t\t",end="->")
        disease_input = input("")
        conf,cnf_dis=check_pattern(chk_dis,disease_input)
        if conf==1:
            print("searches related to input: ")
            for num,it in enumerate(cnf_dis):
                print(num,")",it)
            if num!=0:
                print(f"Select the one you meant (0 - {num}):  ", end="")
                conf_inp = int(input(""))
            else:
                conf_inp=0

            disease_input=cnf_dis[conf_inp]
            break
            # print("Did you mean: ",cnf_dis,"?(yes/no) :",end="")
            # conf_inp = input("")
            # if(conf_inp=="yes"):
            #     break
        else:
            print("Enter valid symptom.")

    while True:
        try:
            num_days=int(input("Okay. From how many days ? : "))
            num_days = 7 if num_days>20 else num_days
            #import pdb;pdb.set_trace()
            break
        except:
            print("Enter valid input.")
    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]

            if name == disease_input:
                val = 1
            else:
                val = 0
            if  val <= threshold:
                recurse(tree_.children_left[node], depth + 1)
            else:
                symptoms_present.append(name)
                recurse(tree_.children_right[node], depth + 1)
        else:
            present_disease = print_disease(tree_.value[node])
            # print( "You may have " +  present_disease )
            red_cols = reduced_data.columns 
            symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]
            # dis_list=list(symptoms_present)
            # if len(dis_list)!=0:
            #     print("symptoms present  " + str(list(symptoms_present)))
            # print("symptoms given "  +  str(list(symptoms_given)) )
            color.write("Are you experiencing any ")#,"KEYWORD")
            symptoms_exp=[]
            for syms in list(symptoms_given):
                inp=""
                print(syms,"? : ",end='')
                while True:
                    inp=input("")
                    if(inp=="yes" or inp=="no"):
                        break
                    else:
                        print("provide proper answers i.e. (yes/no) : ",end="")
                if(inp=="yes"):
                    symptoms_exp.append(syms)

            second_prediction=sec_predict(symptoms_exp)
            # print(second_prediction)
            calc_condition(symptoms_exp,num_days)
            if(present_disease[0]==second_prediction[0]):
                print("You may have ", present_disease[0])
                print()
                print(description_list[present_disease[0]])
                row = doctors[doctors['disease'] == present_disease[0]]
                print()
                print('Consult',str(row['name'].values))
                print()
                print('Visit',str(row['link'].values))
                print()
                # readn(f"You may have {present_disease[0]}")
                # readn(f"{description_list[present_disease[0]]}")

            else:
                print("You may have ", present_disease[0], "or ", second_prediction[0])
                print()
                print(description_list[present_disease[0]])
                print(description_list[second_prediction[0]])
                row = doctors[doctors['disease'] == present_disease[0]]
                print()
                print('Consult',str(row['name'].values))
                print()
                print('Visit',str(row['link'].values))
                print()

            # print(description_list[present_disease[0]])
            precution_list=precautionDictionary[present_disease[0]]
            print("Take following measures : ")
            print()
            for  i,j in enumerate(precution_list):
                print(i+1,")",j)

            # confidence_level = (1.0*len(symptoms_present))/len(symptoms_given)
            # print("confidence level is " + str(confidence_level))

    recurse(0, 1)

In [50]:
doc_dataset = pd.read_csv('doctors_dataset.csv', names = ['Name', 'Description'])


diseases = reduced_data.index
diseases = pd.DataFrame(diseases)

doctors = pd.DataFrame()
doctors['name'] = np.nan
doctors['link'] = np.nan
doctors['disease'] = np.nan

doctors['disease'] = diseases['prognosis']


doctors['name'] = doc_dataset['Name']
doctors['link'] = doc_dataset['Description']

record = doctors[doctors['disease'] == 'AIDS']
record['name']
record['link']
getSeverityDict()
getDescription()
getprecautionDict()
getInfo()
tree_to_code(clf,cols)
#import pdb;pdb.set_trace()
print("----------------------------------------------------------------------------------------")

-----------------------------------HealthCare ChatBot-----------------------------------

Your Name? 				->Hello,  maha

Enter the symptom you are experiencing  		->Enter valid symptom.

Enter the symptom you are experiencing  		->Enter valid symptom.

Enter the symptom you are experiencing  		->searches related to input: 
0 ) high_fever
1 ) mild_fever
Select the one you meant (0 - 1):  Are you experiencing any muscle_weakness ? : stiff_neck ? : swelling_joints ? : movement_stiffness ? : painful_walking ? : It might not be that bad but you should take precautions.
You may have  Arthritis or  Typhoid

Arthritis is the swelling and tenderness of one or more of your joints. The main symptoms of arthritis are joint pain and stiffness, which typically worsen with age. The most common types of arthritis are osteoarthritis and rheumatoid arthritis.
An acute illness characterized by fever caused by infection with the bacterium Salmonella typhi. Typhoid fever has an insidious onset, with fever,

In [51]:
import pickle
with open("svm_model.pkl", "wb") as model_file:
    pickle.dump(model, model_file)

with open("label_encoder.pkl", "wb") as le_file:
    pickle.dump(le, le_file)

In [26]:
import pandas as pd
df=pd.read_csv("Training.csv")
df.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection


In [ ]:
# # app.py: Flask application for healthcare diagnostic system

# from flask import Flask, request, jsonify
# import pickle
# import pandas as pd

# app = Flask(__name__)

# # Load trained models and other assets
# model = pickle.load(open("svm_model.pkl", "rb"))
# label_encoder = pickle.load(open("label_encoder.pkl", "rb"))

# @app.route('/predict', methods=['POST'])
# def predict():
#     try:
#         data = request.json
#         symptoms = pd.DataFrame([data['symptoms']])
#         prediction = model.predict(symptoms)
#         disease = label_encoder.inverse_transform(prediction)
#         return jsonify({"disease": disease[0]})
#     except Exception as e:
#         return jsonify({"error": str(e)}), 400

# if __name__ == '__main__':
#     app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
